In [ ]:
!pip install -q accelerate bitsandbytes
!pip install -q trl py7zr auto-gptq optimum
!pip install datasets -q
!pip install --upgrade peft transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from huggingface_hub import notebook_login
from trl import SFTTrainer
import pandas as pd
import os

base_model = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import pandas as pd
!pip install datasets -q
from datasets import Dataset

json_dataset = []
f = "/content/drive/MyDrive/Colab Notebooks/data.jsonl"
with open(f, 'r') as f:
    for line in f:
        json_dataset.append(json.loads(line))

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.model_max_length = 32768
tokenizer.truncation_side = 'right'
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

LlamaTokenizerFast(name_or_path='TheBloke/Mistral-7B-Instruct-v0.2-GPTQ', vocab_size=32000, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
dataset = [tokenizer.apply_chat_template(story, truncation=True, tokenize=False) for story in json_dataset]
dataset = {'stories': dataset}

df = pd.DataFrame.from_dict(dataset, orient='columns')
dataset = Dataset.from_pandas(df)

print(dataset.column_names)

['stories']


In [ ]:
quantization_config_loading = GPTQConfig(
    bits=4,
    disable_exllama=True,
    tokenizer=tokenizer
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quantization_config_loading,
    device_map="auto"
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model.config

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralConfig {
  "_name_or_path": "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "pretraining_tp": 1,
  "quantization_config": {
    "batch_size": 1,
    "bits": 4,
    "block_name_to_quantize": null,
    "cache_block_outputs": true,
    "damp_percent": 0.1,
    "dataset": null,
    "desc_act": true,
    "exllama_config": {
      "version": 1
    },
    "group_size": 128,
    "max_input_length": null,
    "model_seqlen": null,
    "module_name_preceding_first_block": null,
    "modules_in_block_to_quantize": null,
    "pad_token_id": null,
    "quant_method": "gptq",
    "sym": true,
    "tokenizer":

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ]
)

model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./results",
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=2,
    save_strategy="epoch",
    evaluation_strategy = "no",
    fp16=False,
    max_steps=-1,
    push_to_hub=False,
    group_by_length=False,
)

In [ ]:
trainer = SFTTrainer(
  model=model,
  train_dataset=dataset,
  peft_config=peft_config,
  dataset_text_field="stories",
  args=training_arguments,
  tokenizer=tokenizer,
  packing=False,
  max_seq_length=None
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/582 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in

TrainOutput(global_step=146, training_loss=1.8403111287992295, metrics={'train_runtime': 4401.5318, 'train_samples_per_second': 0.264, 'train_steps_per_second': 0.033, 'total_flos': 1243371490246656.0, 'train_loss': 1.8403111287992295, 'epoch': 2.0})

In [ ]:
notebook_login()

In [ ]:
model.push_to_hub("Mistral-7B-Instruct-v0.2-PEFT-adapters-v2")

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tariq9mehmood9/Mistral-7B-Instruct-v0.2-PEFT-adapters-v2/commit/7b549b74422f5919212fe378153cfbca7259350e', commit_message='Upload model', commit_description='', oid='7b549b74422f5919212fe378153cfbca7259350e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (rotary_emb): MistralRotaryEmbedding()
              (k_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (quant_linear_module): QuantLinear()
          

In [ ]:
from peft import PeftModel
adapters_name = 'tariq9mehmood9/Mistral-7B-Instruct-v0.2-PEFT-adapters-v2'

model_base = AutoModelForCausalLM.from_pretrained(
    adapters_name,
    device_map="auto",
    trust_remote_code=True,
    revision="main"
)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained(base_model)
pipe = pipeline(
    task='text-generation',
    model=model_base,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True, # creative generation by discouraging greedy decoding
    temperature=0.75,
    top_p=0.95,
    return_full_text = False  # Only return the current output instead of returning complete prompt
    )

def parse_first_json(json_str):
    end_index = json_str.find("}") # Find the index of the first closing curly brace
    if end_index != -1:  # If a closing brace is found
        first_json_obj = json_str[:end_index + 1] # Parse the substring containing the first JSON object
        parsed_json = json.loads(first_json_obj) # Parse the JSON object
        return parsed_json
    else:
        return None

story = ""

for _ in range(10):

  system = f"""
  Starting from now, assume the role of an expert interactive fiction writer who specializes in crafting short, engaging, and captivating multiple-choice narratvies. \
  Your task is to generate a small narrative of no more than 50 words continuing the story so far provided to you in the square brackets. \
  Give 3 choices, that is, 3 possible continuations of the story for the current narrative. Every time you must only return a single JSON object with the follwoing three keys strictly following the provided description:
  {{
  "story": string, current generated narrative continuing the past story provided below in square brackets.
  "options": list, a list containing three string-type elements corresponding to the 3 options.
  }}
  Do not generate anything other than the JSON object inside {{}}.
  """

  assistant = f"""
  {{"story": "As you flee from the cursed room, you hear the mummy's voice growing louder. Sweat pouring down your face, you make a hasty decision.", "options": ["Seek help from the village elder.", "Search for a way to break the curse.", "Abandon your quest and leave the relics behind."]}}
  """

  user = f"""
  Start a new story if the story given below in the square brackets is empty, otherwise continue the story so far following the rules provided earlier.

  [{story}]

  """

  # one-shot prompting
  chat = [
    {"role": "user", "content": system},
    {"role": "assistant", "content": assistant},
    {"role": "user", "content": user},
    {"role": "assistant", "content": "```json\n{"},
  ]
  chat = [
      {"role": "user", "content": "Starting from now, assume the role of an expert interactive fiction writer who specializes in crafting short, engaging, and captivating narratvies. You are tasked with generetaing a small narrative in the world of Harry Potter. Your response should not exceed 50 words."}
  ]

  # prepare the prompt using the chat template
  prompt = tokenizer.apply_chat_template(chat, tokenize=False)
  # run the pipeline to generate the model output
  outputs = pipe(prompt)
  output = outputs[0]["generated_text"].strip()
  print(output)

  # sometimes the model generates notes/explanation after json so we only parse the first json object
  break


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You've done it. You've broken into Gryffindor. You've found the Sword of Gryffindor, the legendary artifact that will save Hogwarts from the clutches of Voldemort. You've beaten the rival Slytherin students to it.

You've made it.

You've made it. You've found the Sword of Gryffindor, the legendary artifact that will save Hogwarts from the clutches of Voldemort. You've beaten the rival Slytherin students to it.

You've made it. You've found the Sword of Gryffindor, the legendary artifact that will save Hogwarts from the clutches of Voldemort. You've beaten the rival Slytherin students to it.

You've made it. You've found the Sword of Gryffindor, the legendary artifact that will save Hogwarts from the clutches of Voldemort. You've beaten the rival Slytherin students to it.

You've made it. You've found the Sword of Gryffindor, the legendary artifact that will save Hogwarts from the clutches of Voldemort. You've beaten the rival Slytherin students to it.

You've made it. You've found the

In [ ]:
print(prompt)

<s>[INST] Starting from now, assume the role of an expert interactive fiction writer who specializes in crafting short, engaging, and captivating narratvies. [/INST]
